In [ ]:
### Saurabh - 4th July 2023 - Adhoc notebook for All of u
#### saurabhmangal@google.com

## V2- New updates on 4&5th July -made the prompts works by adding token length and using prompting
## V3- includes text-bison@latest model -- look for example 6,7,8

## Download csv

In [ ]:
!gsutil cp -r 'gs://my-project-0004-bucket02/llms/Sample Pork No Pork SKUs.csv' ./


Copying gs://my-project-0004-bucket02/llms/Sample Pork No Pork SKUs.csv...
/ [1 files][ 11.6 KiB/ 11.6 KiB]                                                
Operation completed over 1 objects/11.6 KiB.                                     


In [ ]:
# import io
# import pandas as pd

# Create a Pandas DataFrame from the CSV data
# df_retail = pd.read_csv('Sample Pork No Pork SKUs.csv')

In [ ]:
# df_retail.columns

In [ ]:
# df_retail['product_name'].unique()

## V1 version using Serapapi API

In [ ]:
# I'm storing my api keys in a local secrets folder
# Feel free to use whatever setup works for you

import os
os.environ["SERPAPI_API_KEY"] = ""

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

from langchain.llms import VertexAI

# First, let's load the language model we're going to use to control the agent.
llm = VertexAI(temperature=0.2)

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
res = agent.run(''' You are a helpful content creator for products in a grocery store.

                Create a Product description for following product:  Heinz Beanz Baked Beans - Original

''')



> Entering new AgentExecutor chain...
 The product is Heinz Beanz Baked Beans - Original. 
Action: Search
Action Input: Heinz Beanz Baked Beans - Original
Observation: Heinz Baked Beans are a brand of baked beans produced by the H.J. Heinz Company. They have been sold as "Heinz Beanz" in the United Kingdom since 2008.
Thought: Heinz Baked Beans are a brand of baked beans produced by the H.J. Heinz Company. 
Action: Search
Action Input: Are Heinz Baked Beans vegan?
Observation: Are all baked beans suitable for vegans? You've specifically been asking 'are Heinz baked beans vegan' and 'are Branston baked beans vegan'? Well, yes — they're both vegan too! Send more vegan questions our way in the comments section.
Thought: Heinz Baked Beans are a brand of baked beans produced by the H.J. Heinz Company. They are vegan. 
Action: Search
Action Input: What are the ingredients in Heinz Beanz Baked Beans - Original?
Observation: Trending
Thought: The ingredients in Heinz Beanz Baked Beans - Orig

## V2 Scenario with Enhanced Results -- With Google Search API (not Serapapi)

In [ ]:


import os

os.environ["GOOGLE_CSE_ID"] = ""
os.environ["GOOGLE_API_KEY"] = "---"


In [ ]:
from langchain.tools import Tool
from langchain.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()

tools = Tool(
    name="Google Search",
    description="Search Google for recent results.",
    func=search.run,
)

In [ ]:
tools.run("ingredients of Heinz Beanz Baked Beans - Original?")



'May 11, 2020 ... Navy beans / haricot, if you can find them (that\'s what Heinz uses) otherwises any beans (preferably white). Ingredients. Cups Metric. Beans –\xa0... Heinz Baked Beans are a brand of baked beans produced by the H.J. Heinz Company. They have been sold as "Heinz Beanz" in the United Kingdom since 2008. Amazon\'s Choice. Heinz Beanz, Organic Baked Beans in Tomato Sauce, 415 Gram ... Ingredient Type, \u200eNo Artificial Colors. Diet Type, \u200eVegetarian. Ingredients · Beans (50%), · Tomatoes (36%), · Water, · Sugar, · Spirit Vinegar, · Modified Cornflour, · Salt, · Spice Extracts,\xa0... 1 of your 5 a day and naturally high in protein and fibre · No artificial colours, flavours or preservatives · Suitable for Vegetarians and Vegans · Gluten free\xa0... Jul 3, 2023 ... If you look at the label of a tin of Heinz beans, vinegar is listed as a separate ingredient and if yours tasted sour, it is possible there was\xa0... Jul 14, 2022 ... Of course, that sweet-savory combina

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

from langchain.llms import VertexAI

# First, let's load the language model we're going to use to control the agent.
# llm = VertexAI(temperature=0.2)
llm = VertexAI(temperature=0.2,model_name='text-bison@001')

# llm = VertexAI(temperature=0.2,model_name='text-bison@latest')

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
tools = load_tools(["google-search", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!


In [ ]:
prompt_002 = ''' In this task, you will be an expert content creator for products in a grocery store. You are also able to classify if some of the ingredients contains Pork, Lard, which out musilm friends may not like

Your task is then to extract the information in a JSON format for it to be passed to another system, hence the formatting rules for each of the fields listed below needs to be followed very strictly. In the case of the prompt, you should be search on Google to figure these inputs.
The required fields to be used as the JSON keys are as follows:

1. is_halal	- 0 if it doesnt contain Pork or is not halal or 1 based on if it contains Pork, Pig , if not sure give Not sure.
2. is_organic - check the ingredient list to see if its organic
3. is_vegeterian - check the ingredient list to see if its contains any kind of meat or animal oils.
4. is_healthier_choice	- check the ingredient list to see if its organic

Create a Product description for following product here:  Beksul Luncheon Meat - Original


'''


In [ ]:
res = agent.run(prompt_002)



> Entering new AgentExecutor chain...
I need to find out if Beksul Luncheon Meat - Original is halal
Action: google_search
Action Input: Beksul Luncheon Meat - Original is halal
Observation: Golden Bridge Chicken Luncheon Meat - Original. $4.20$4.54 ... Golden Palm Ham Luncheon Meat - Premium Chicken ... 340g•Halal. 4.5 (11). Add to cart. CJ Beksul pancake mix is a ready pancake mix flour with no need for extra ... Halal Chicken Luncheon Meat 340g Original | Cheese Luncheon Meat | Black ... DONGWON LUNCHEON MEAT 340g Non Halal Canned Food Johor Bahru (JB), Malaysia, Skudai Supplier, Suppliers, Supply, Supplies, HONG FONG GROUP TRADING - Based ... ... Jjawang Jjajang Noodle with Black Bean Paste (Piece) Made in Korea [Flavors]: -Original Non Spicy -Hot & Spicy -Original Non Frying ○WHAT'S JJAWANG: 1. Mili Corned Beef. $4.64. Mili Corned Beef. 340g•Halal ... Golden Bridge Pork Luncheon Meat - Original. $4.20$4.54 ... Beksul Luncheon Meat. $3.95$6.56. Since 1981, Beksul luncheon meant h

## Example #2

In [ ]:
prompt_002 = ''' In this task, you will be an expert content creator for products in a grocery store. You are also able to classify if some of the ingredients contains Pork, Lard, which out musilm friends may not like

Your task is then to extract the information in a JSON format for it to be passed to another system, hence the formatting rules for each of the fields listed below needs to be followed very strictly. In the case of the prompt, you should be search on Google to figure these inputs.
The required fields to be used as the JSON keys are as follows:

1. is_halal	- 0 if it doesnt contain Pork or is not halal or 1 based on if it contains Pork, Pig , if not sure give Not sure.
2. is_organic - check the ingredient list to see if its organic
3. is_vegeterian - check the ingredient list to see if its contains any kind of meat or animal oils.
4. is_healthier_choice	- check the ingredient list to see if its organic
5. New_categories -

Create a Product description for following product here:  TungLok Zha Jiang Noodle 390 gm

Do step by step process by first searching the ingredients

'''


In [ ]:
res = agent.run(prompt_002)



> Entering new AgentExecutor chain...
I need to find the ingredients of TungLok Zha Jiang Noodle 390 gm
Action: google_search
Action Input: TungLok Zha Jiang Noodle 390 gm
Observation: Shop for TungLok Zha Jiang Noodle 390 G from Singapore's trusted grocery retailer. FairPrice offers a wide range of products with prices matched online and ... TungLok Home Fiesta Zha Jiang Mian - Frozen. $15.60. 5.0(3)·32 sold. Singapore. TungLok Frozen Singapore. frozen sweet potatoes fries. Ready Meals Ready to ... TungLok Home Fiesta Zha Jiang Mian - Frozen. 380g. $15.60. ADD TO CART. TungLok ... BEIJING JINGHUA CORP. OF FOREIGN TRADE & ECONOMIC COOP. 15, HUAYUAN LU, BEITAIPINGZHUANG, HAIDIAN DISTRICT, BEIJIANG, CHINA. CNBE0022, BEIJING LONG QI FOOD CO ... ... 390 EAST ZHONGSHAN ROAD NINGBO, CHINA. 869, CNCI0034, CIXI YONGCHING FROZEN FOODS CO ... JIANG YANG INSTRIAL ZONE, YANGZHOU, JIANGSU, CHINA. 7199, CNYA0192 ... Aug 15, 2020 ... ... 390 AMIT SAH. Male. 09405811. प्रदेश नं. २. धनुषा ... noodle 

In [ ]:
prompt_002 = ''' In this task, you will be an expert content creator for products in a grocery store. You are also able to classify if some of the ingredients contains Pork, Lard, which out musilm friends may not like

Your task is then to extract the information in a JSON format for it to be passed to another system, hence the formatting rules for each of the fields listed below needs to be followed very strictly. In the case of the prompt, you should be search on Google to figure these inputs.
The required fields to be used as the JSON keys are as follows:

1. is_halal	- 0 if it doesnt contain Pork or is not halal or 1 based on if it contains Pork, Pig , if not sure give Not sure.
2. is_organic - check the ingredient list to see if its organic  , if not sure give Not sure.
3. is_vegeterian - check the ingredient list to see if its contains any kind of meat or animal oils  , if not sure give Not sure.
4. is_healthier_choice	- check the ingredient list to see if its organic  , if not sure give Not sure.

Create a Product description for following product here: Aw's Market Baby Honey Baked Ham (Whole)


'''


In [ ]:
res = agent.run(prompt_002)



> Entering new AgentExecutor chain...
I need to find out if the product contains pork or not
Action: google_search
Action Input: Aw's Market Baby Honey Baked Ham (Whole)
Observation: Locally smoked and/or cured. This baby honey baked ham is a heft of sweet deliciousness at 1 kilogram. No longer be limited by sliced ham! Cube it, chop it, ... Buy 1 Aw's Market Baby Honey Baked Ham (Whole) @ $12.80. Grab now before 1 Sep 2023! Save $19.20. Aw's Market Baby Honey Baked Ham (Whole). $12.80$32.00. Aw's Market Baby Honey Baked Ham (Whole). 1000 G. 4.4 (10). By Tue, 22 Aug. Add to cart. Rudi's Honey Baked Ham (Sliced). $5.90. Rudi's Honey Baked Ham (Sliced). 200 G. Add to cart · Spend $50 Save $3. +1. Aw's Market Baby Honey Baked Ham (Whole). Save $19.20. Aw's Market Baby Honey Baked Ham (Whole). $12.80 ; Save $3.15. Willms Sausage, Bockwurst, Willms, GERMAN. $7.35 ; Save $6.12. Aw's Market Honey Baked ... Get your groceries from Aw's Market (Punggol) delivered to your door in no time! ... 

## Example #4 (includes pork)

In [ ]:


prompt_002 = ''' In this task, you will be an expert content creator for products in a grocery store. You are also able to classify if some of the ingredients contains Pork, Lard, which out musilm friends may not like

Your task is then to extract the information in a JSON format for it to be passed to another system, hence the formatting rules for each of the fields listed below needs to be followed very strictly. In the case of the prompt, you should be search on Google to figure these inputs.
The required fields to be used as the JSON keys are as follows:

1. is_halal	- 1 if it doesnt contain Pork or is not halal or 0 based on if it contains Pork, Pig , if not sure give Not sure.
2. is_organic - check the ingredient list to see if its organic  , if not sure give Not sure.
3. is_vegeterian - check the ingredient list to see if its contains any kind of meat or animal oils  , if not sure give Not sure.
4. is_healthier_choice	- check the ingredient list to see if its organic  , if not sure give Not sure.

Create a Product description for following product here: Beksul Luncheon Meat


'''


In [ ]:
res = agent.run(prompt_002)



> Entering new AgentExecutor chain...
I need to find out the ingredients of Beksul Luncheon Meat
Action: google_search
Action Input: Beksul Luncheon Meat
Observation: Buy 1 Beksul Luncheon Meat @ $3.95. Grab now before 31 Aug 2023! Cj Beksul Luncheon Meat Pork Can 200g / 340g ... Bring home the timeless lunchmeat lovers. Ready to be served as is or used for cooking, this classic lunchtime ... CJ Beksul Luncheon Meat is canned ham with a difference. Ready to be served as is or used for cooking, this classic lunchtime staple will surely find a ... CJ Beksul Luncheon Meat is canned ham with a difference. Ready to be served as is or used for cooking, this classic lunchtime staple will surely find a ... Korean Luncheon Meat Can 340g Product of Korea. Net Weight340g (715kcal)HighlightsSince 1981 , Beksul Luncheon Meat has made convenient life and enjoyable table for you.IngredientsPork 45.63%, Chicken ... The luncheon meat is a soft and great texture, suitable for side dishes or sandwiches

OutputParserException: Could not parse LLM output: `I found that the ingredients of Beksul Luncheon Meat is Pork 45.63%, Chicken 14.27%, Water 20.00%, Salt 1.00%, Sugar 1.00%, Sodium Nitrite 0.01%, Sodium Ascorbate 0.01%, Sodium Tripolyphosphate 0.01%, Sodium Diacetate 0.01%, Disodium Inosinate 0.01%, Disodium Guanylate 0.01%, Sodium Erythorbate 0.0`

### Example #5 (may includes pork)  and this the main prompt

In [ ]:


prompt_002 = ''' In this task, you will be an expert content creator for products in a grocery store. You are also able to classify if some of the ingredients contains Pork, Lard, which out musilm friends may not like

Your task is then to extract the information in a JSON format for it to be passed to another system, hence the formatting rules for each of the fields listed below needs to be followed very strictly. In the case of the prompt, you should be search on Google to figure these inputs.
The required fields to be used as the JSON keys are as follows:

1. is_halal	- 1 if it doesnt contain Pork or is not halal or 0 based on if it contains Pork, Pig ,Lard and if not sure give Not sure.
2. is_organic - check the ingredient list to see if its organic  , if not sure give Not sure.
3. is_vegeterian - check the ingredient list to see if its contains any kind of meat or animal oils  , if not sure give Not sure.
4. is_healthier_choice	- check the ingredient list to see if its organic  , if not sure give Not sure.
5. is_allegant -

Create a Product description for following product here: Braised Yee Fu Noodles



'''


## Zhajiangmian
# Braised Yee Fu Noodles

In [ ]:
res = agent.run(prompt_002)



> Entering new AgentExecutor chain...
I need to find out the ingredients of Braised Yee Fu Noodles
Action: google_search
Action Input: Braised Yee Fu Noodles ingredients
Observation: Ingredients · 1 lb yee mein noodles · 1 bunch yellow chives · 8 dried shiitake mushrooms · 2 king oyster mushroom · 1 medium carrot · 1 clove garlic. Nov 15, 2016 ... Mix all the ingredients for the sauce in a bowl · Wash white chives and drain · Cut chives about 3 inches in length · Wash straw mushrooms and cut ... Sep 23, 2014 ... This is a restaurant quality recipe for braised ee-fu noodles with boneless roasted duck and lots of mushrooms. Satisfaction guaranteed! Jun 22, 2011 ... Ingredients · 200g e-fu noodles · 150g yellow chives · 150g fresh straw mushrooms · 2 cloves garlic, finely chopped · 1/2 tsp cooking wine · 1 tbsp ... Apr 19, 2021 ... Ingredients · 6 dried shiitake mushrooms, soaked in boiling water for 10 minutes · 120 gm fresh enoki mushrooms, woody ends trimmed · 200 gm yi-fu ... May 2,

## Example #6

In [ ]:
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

from langchain.llms import VertexAI

# First, let's load the language model we're going to use to control the agent.
llm = VertexAI(temperature=0.2)

search = GoogleSearchAPIWrapper()

tool = Tool(
    name="Google Search",
    description="Search Google for recent results.",
    func=search.run,
)

In [ ]:
ingredients_list = tool.run("ingredients in  Nissin Instant Japanese Ramen - Kyushu Black ?")

print(ingredients_list)

Wheat flour (contains iron, zinc, vitamins [B2, B1, folic acid]) (69%), palm oil (antioxidants [307b, 320, 321]), tapioca starch, salt, acidity regulators (501, ... Aug 11, 2014 ... Kyushu ramen is known for it's cloudy broth and deep, rich flavors. Nissin Kyushu Black includes black garlic oil! Products 1 - 6 of 6 ... Ingredients: Noodles: Wheat Flour (Contains Iron, Zinc, Vitamin (B2, B1, Folic Acid) (70%), Palm Oil (Antioxidants (E307b, E320, E321)), ... Storage. Store in a cool, dry place ; Ingredients · Noodles: Wheat Flour, Palm Oil, Tapioca Starch, Salt, Acidity Regulators, Colour, Stabiliser, Emulsifier ... Sep 30, 2013 ... And we truly appreciate ramen in the form of instant noodles. Imagine, your favorite food at the ... Nissin's Kyushu Black Japanese Ramen. Japan Nissin Best Selling instant ramen, a bowl of enriched broth soup ramen brings you a real and authentic taste of Japan. Kyushu Black Ramen. - Thick creamy ... Aug 16, 2018 ... He has partnered Japanese food manufactu

In [ ]:
from langchain import PromptTemplate, LLMChain

template = """Question: {question}

Answer: Let's think step by step. and convert the list into a clean json file"""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm = VertexAI()

llm_chain = LLMChain(prompt=prompt, llm=llm)

# question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

ingredients_list_json = llm_chain.run(ingredients_list)

In [ ]:


prompt_003 = ''' In this task, you will be an expert content creator for products in a grocery store. You are also able to classify if some of the ingredients contains Pork, Lard, which out musilm friends may not like

Your task is then to extract the information in a JSON format for it to be passed to another system, hence the formatting rules for each of the fields listed below needs to be followed very strictly. In the case of the prompt, you should be search on Google to figure these inputs.
The required fields to be used as the JSON keys are as follows:

1. is_halal	- 1 if it doesnt contain Pork or is not halal or 0 based on if it contains Pork, Pig ,Lard and if not sure give Not sure.
2. is_organic - check the ingredient list to see if its organic  , if not sure give Not sure.
3. is_vegeterian - check the ingredient list to see if its contains any kind of meat or animal oils  , if not sure give Not sure.
4. is_healthier_choice	- check the ingredient list to see if its organic  , if not sure give Not sure.

Create a Product description for following product here: Braised Yee Fu Noodles

and here and the ingredients list =

'''  + ingredients_list_json #+ ''' only give the final json file'''




In [ ]:
prompt_003

' In this task, you will be an expert content creator for products in a grocery store. You are also able to classify if some of the ingredients contains Pork, Lard, which out musilm friends may not like \n                \nYour task is then to extract the information in a JSON format for it to be passed to another system, hence the formatting rules for each of the fields listed below needs to be followed very strictly. In the case of the prompt, you should be search on Google to figure these inputs. \nThe required fields to be used as the JSON keys are as follows:\n                \n1. is_halal\t- 1 if it doesnt contain Pork or is not halal or 0 based on if it contains Pork, Pig ,Lard and if not sure give Not sure.\n2. is_organic - check the ingredient list to see if its organic  , if not sure give Not sure.\n3. is_vegeterian - check the ingredient list to see if its contains any kind of meat or animal oils  , if not sure give Not sure. \n4. is_healthier_choice\t- check the ingredient 

In [ ]:
from langchain import PromptTemplate, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm = VertexAI()

llm_chain = LLMChain(prompt=prompt, llm=llm)

# question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

llm_chain.run(prompt_003)

' ```json\n{\n  "name": "Braised Yee Fu Noodles",\n  "ingredients": [\n    "Noodles: Wheat Flour (Contains Iron, Zinc, Vitamin (B2, B1, Folic Acid) (70%), Palm Oil (Antioxidants (E307b, E320, E321)), Tapioca Starch, Salt, Acidity Regulators, Colour, Stabiliser, Emulsifier",\n    "Soup: Pork Bone Broth, Black Garlic Oil, Soy Sauce, Miso, Salt, Sugar, Spices, Flavour Enhancers (E62'

## Example #7


In [ ]:
from langchain.llms import VertexAI


llm = VertexAI(model_name='text-bison@latest')
question = "What day comes after Friday?"
llm(question)

' Saturday'

## Example #8

In [ ]:
import vertexai
from vertexai.language_models import TextGenerationModel

vertexai.init(project="my-project-0004-346516", location="us-central1")
parameters = {
    "max_output_tokens": 2048,
    "temperature": 0.2,
    "top_p": 0.8,
    "top_k": 40
}

# model = TextGenerationModel.from_pretrained("text-bison@001")
model = TextGenerationModel.from_pretrained("text-bison@latest")

response = model.predict(
    ''' In this task, you will be an expert content creator for products in a grocery store.

1. is_halal	- 0 if it doesnt contain Pork or is not halal or 1 based on if it contains Pork, Pig , if not sure give Not sure.

Create a indicator is_halal in json format for following product here:  Beksul Luncheon Meat - Original

''',
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model:  ```json
{
  "is_halal": "Not sure"
}
```
